In [1]:
import pandas as pd
from pandas.io import sql
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [2]:
import sqlite3

In [3]:
#This creates a database that we are connected to, but we have no data, at the moment this is empty.
conn = sqlite3.connect('dat-test.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [4]:
#We read in the data from CSV
data = pd.read_csv('dataset/rossmann_sales.csv', low_memory=False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
#This links the dataframe to our SQLlite database
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [6]:
sql.read_sql("select * from rossmann_sales where customers > 500 limit 10", con=conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [7]:
new_data = pd.read_csv('dataset/rossmann_sales.csv', low_memory=False)
new_data.to_sql("new_table", con=conn, if_exists='replace', index=False)

> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [8]:
sql.read_sql("select store, date, customers from new_table limit 10", con=conn)

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559
5,6,2015-07-31,589
6,7,2015-07-31,1414
7,8,2015-07-31,833
8,9,2015-07-31,687
9,10,2015-07-31,681


In [9]:
sql.read_sql("select store, date, sales, customers from new_table where sales > 0 group by date limit 25", con=conn)

,Store,Date,Sales,Customers
0,1097,2013-01-01,5961,1405
1,1115,2013-01-02,3697,305
2,1115,2013-01-03,4297,300
3,1115,2013-01-04,4540,326
4,1115,2013-01-05,4771,339
5,1097,2013-01-06,8258,2003
6,1115,2013-01-07,6905,471
7,1115,2013-01-08,5243,341
8,1115,2013-01-09,4649,324
9,1115,2013-01-10,5007,339


> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [10]:
sql.read_sql("select store, date, customers from new_table where open = 1 and promo = 1 group by store", con=conn)

,Store,Date,Customers
0,1,2013-01-07,785
1,2,2013-01-07,763
2,3,2013-01-07,1079
3,4,2013-01-07,1562
4,5,2013-01-07,717
5,6,2013-01-07,947
6,7,2013-01-07,1150
7,8,2013-01-07,802
8,9,2013-01-07,605
9,10,2013-01-07,751


> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [11]:
sql.read_sql("select sum(sales) from new_table where promo = 1", con=conn)

,sum(sales)
0,3101206286


#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [12]:
#Loading the data
wal_conn = sqlite3.connect('wal-mart.db')
walmart = pd.read_csv('dataset/walmart-sales.csv', low_memory=False)
#Table for data
walmart.to_sql('waldata',
            con=wal_conn,
            if_exists='replace',
            index=False)
#Table for features
walfeatures = pd.read_csv('dataset/features.csv', low_memory=False)
walfeatures.to_sql('walfeatures',
            con=wal_conn,
            if_exists='replace',
            index=False)

In [13]:
sql.read_sql("select * from walfeatures limit 5", con=wal_conn)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,None,None,None,None,None,211.096358,8.106,0
1,1,2010-02-12,38.51,2.548,None,None,None,None,None,211.242170,8.106,1
2,1,2010-02-19,39.93,2.514,None,None,None,None,None,211.289143,8.106,0
3,1,2010-02-26,46.63,2.561,None,None,None,None,None,211.319643,8.106,0
4,1,2010-03-05,46.50,2.625,None,None,None,None,None,211.350143,8.106,0


In [14]:
sql.read_sql("select * from waldata limit 5", con=wal_conn)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,0
1,1,1,2010-02-12,46039.49,1
2,1,1,2010-02-19,41595.55,0
3,1,1,2010-02-26,19403.54,0
4,1,1,2010-03-05,21827.90,0


In [15]:
#Select the store, date and fuel price on days it was over 90 degrees
sql.read_sql("select s.store, s.date, f.fuel_price from waldata as s join walfeatures as f on s.store = f.store "\
             "and s.date = f.date where temperature>90 limit 10", con=wal_conn)

,Store,Date,Fuel_Price
0,1,2011-08-05,3.684
1,1,2011-08-05,3.684
2,1,2011-08-05,3.684
3,1,2011-08-05,3.684
4,1,2011-08-05,3.684
5,1,2011-08-05,3.684
6,1,2011-08-05,3.684
7,1,2011-08-05,3.684
8,1,2011-08-05,3.684
9,1,2011-08-05,3.684


In [16]:
#Select the store, date and weekly sales and temperature
sql.read_sql("select s.store, s.date, s.weekly_sales, f.temperature from waldata as s join walfeatures as f "\
             "on s.store = f.store and s.date = f.date limit 10",con=wal_conn)

,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50
5,1,2010-03-12,21043.39,57.79
6,1,2010-03-19,22136.64,54.58
7,1,2010-03-26,26229.21,51.45
8,1,2010-04-02,57258.43,62.27
9,1,2010-04-09,42960.91,65.86


In [17]:
#What were average sales on holiday vs. non-holiday sales
sql.read_sql("select isholiday, avg(weekly_sales) from waldata group by IsHoliday",con=wal_conn)

,IsHoliday,avg(weekly_sales)
0,0,15901.445069
1,1,17035.823187


In [18]:
#What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
sql.read_sql("select s.isholiday, avg(s.weekly_sales) from waldata as s join walfeatures as f "\
             "on s.date = f.date and s.store = f.store where f.temperature<32 group by s.IsHoliday",con=wal_conn)

,IsHoliday,avg(s.weekly_sales)
0,0,15275.770307
1,1,15111.471238


In [19]:
#Unemployment
unemp = sql.read_sql("select s.date, s.weekly_sales, f.CPI, f.unemployment from waldata as s join walfeatures as f "\
             "on s.date = f.date and s.store = f.store group by s.date limit 100",con=wal_conn)
unemp = pd.DataFrame(unemp)

In [ ]:
unemp.head()

In [ ]:
unemp.plot.scatter(x="Weekly_Sales",y="Unemployment", s=1.5**unemp["CPI"])